In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv("C:/Users/hp/Desktop/pandas/StudentsPerformance.csv")

data = shuffle(data)

feature = data.drop(["gender"],axis=1)
feature = pd.get_dummies(feature)

label = data["gender"]
classes = label.unique()
label = pd.get_dummies(label)

In [3]:
features = np.array(feature)
labels = np.array(label)

In [4]:
features = ss().fit_transform(features)
features[0]
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)
labels.shape

torch.Size([1000, 2])

In [5]:
model = torch.nn.Sequential(
    torch.nn.Linear(18,10),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(10,8),
    torch.nn.ReLU(),
    torch.nn.Linear(8,2),
    torch.nn.Softmax(dim=-1)
)

In [6]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01)

In [8]:
epochs = 0

for epoch in range(epochs):
    preds = model(features)
    loss = loss_fn(preds, labels)

    loss.backward()

    opt.step()
    opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0

            for i in range(len(preds)):
                j = torch.where(preds[i]==preds[i].max())
                try:
                    if labels[i][j] == 1:
                        right += 1
                except:
                    right+=1

            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ len(preds), 1)}%")

In [ ]:
# torch.save(model.state_dict(),"students_weights.pth")

In [10]:
model.load_state_dict(torch.load("students_weights.pth"))

<All keys matched successfully>

In [11]:
preds = model(features)
right = 0
for i in range(len(preds)):
    j = torch.where(preds[i]==preds[i].max())
    try:
        if labels[i][j] == 1:
            right += 1
    except:
            right+=1

            
print(f"Accuracy : {round(right * 100/ len(preds), 1)}%")

Accuracy : 94.8%


In [12]:
index = 121
pred = model(features[index])
pred_i = torch.where(pred == pred.max())
real_i = torch.where(labels[index] == 1)

print(pred, pred_i)

print(f"Pred : {classes[pred_i]}, Actual : {classes[real_i]}")

tensor([0.3241, 0.6759], grad_fn=<SoftmaxBackward0>) (tensor([1]),)
Pred : female, Actual : male
